# CSCI 270 - Computational Humanities
# Lab 3 - Document Clustering

In [ ]:
from scipy.cluster import hierarchy
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import os
import re
from typing import *
from numpy.linalg import norm 

# Setup

Add the CSCI 270 Books and Poems data sets to this notebook's data.

As we develop functions to enable us to perform data clustering, we will use the dictionary given below. Later on, once our functions are working, we will load our documents from files into a dictionary with the same setup, where the keys are filenames (serving as document titles) and the values are the documents themselves.

In [ ]:
filename2document = {"A": "wet snow falls in winter snow falls", 
                     "B": "cold and wet in winter", 
                     "C": "cold wet snow packs best best", 
                     "D": "cold snow is cold and wet"}

The `assess()` function returns `True` if its parameters match, and `False` otherwise. It also prints out the first value. We will use this function to test each of the functions we write.

In [ ]:
def assess(value, expected):
    print(value)
    return value == expected

## Lab 2 Functions

In the code block below, copy over the following functions you wrote in Lab 2:
* `file_dictionary()`
* `all_tokens_from()`
* `count()`
* `count_all()`

In [ ]:
def file_dictionary(file_path: str) -> Dict[str,str]:
    dict = {}
    for file in os.listdir(file_path):
        dict[file] = open(file_path + '/' +  file).read()
    return dict

def all_unique_tokens_from(text: str) -> List[str]:
    unique = []
    for word in all_tokens_from(text):
        if word not in unique:
            unique.append(word)
    return unique
    
def all_tokens_from(text: str) -> List[str]:
    token = text.replace('.', '').lower().split()
    return token

def count(histogram: Dict[Hashable,int], item: Hashable):
    if item not in histogram:
        histogram[item] = 1
    else:
        histogram[item]+=1
    return
    
def count_all(items: Iterable[Hashable]) -> Dict[Hashable,int]:
    return {word:items.count(word) for word in items}

# Part 1 - Finding tf-idf matrix

Write a function to find the term frequencies for all documents. It should return a dictionary, where the key is the name of the file, and the value is a dictionary of term frequency counts.

In [ ]:
def term_freqs(data: Dict[str,str]) -> Dict[str,Dict[str,int]]:
    count_dis = {}
    for filename, contents in data.items():
        temp = {}
        count_dis[filename] = temp
        for word in all_tokens_from(contents):
            if word in temp:
                temp[word] += 1
            else:
                temp[word] = 1
    return(count_dis)

In [ ]:
tf = term_freqs(filename2document)
assess(tf, {'A': {'wet': 1, 'snow': 2, 'falls': 2, 'in': 1, 'winter': 1},
            'B': {'cold': 1, 'and': 1, 'wet': 1, 'in': 1, 'winter': 1},
            'C': {'cold': 1, 'wet': 1, 'snow': 1, 'packs': 1, 'best': 2},
            'D': {'cold': 2, 'snow': 1, 'is': 1, 'and': 1, 'wet': 1}})

Write a function to return a **sorted** list of all words in the corpus. By using an object of the $set()$ class as an intermediate step, we facilitate fast lookup when checking for uniqueness.

In [ ]:
def all_terms(filename2document: Dict[str,Dict[str,int]]) -> List[str]:
    term_list = set()
    for filename, contents in filename2document.items():
        term_list.add(contents)
    term_list = ' '.join(term_list)
    term_list = all_unique_tokens_from(term_list)
    term_list = sorted(term_list)
    return term_list
#    set_list = []
#    for name, words in filename2document.items():
#        for item in words.split():
#            print(item)
#            if item not in set_list:
#                count = 0
#                if len(set_list) == 0 :
#                    set_list.append(item)
#                for wrd in set_list:
#                    if wrd > item:
#                        continue
#                    if wrd < item:
#                        set_list.insert(count, item)
#                    count += 1
#            else:
#                continue
#    return set_list

In [ ]:
terms = all_terms(filename2document) 
assess(terms, ['and', 'best', 'cold', 'falls', 'in', 'is', 'packs', 'snow', 'wet', 'winter'])

Write a function to return a dictionary where the keys are the words in the corpus and the values are the inverse document frequency for that term. Again, to make this efficient, make use of the $set()$ class. In the formula below, $N$ is the total number of documents in the collection, and $|\{d \in D : t \in d\}|$ is the number of documents in which the term $t$ appears.

$$idf(t, D) = log\frac{N}{|\{d \in D : t \in d\}|}$$

In [ ]:
def inv_doc_freqs(data: Dict[str,str], terms: List[str]) -> Dict[str,float]:
    dis_list = []
    for i in data.values():
        dis_list.append(set(all_tokens_from(i)))
    N = len(data)
    finale = {}
    for x in terms:
        num = 0
        for y in dis_list:
            if x in y:
                num += 1
        if num > 0:
            idf = math.log(N/num)
        else:
            idf = 0.0
        finale[x]=idf
    return finale

In [ ]:
idf = inv_doc_freqs(filename2document, terms)
assess(idf,
       {'and': 0.6931471805599453,
        'best': 1.3862943611198906,
        'cold': 0.28768207245178085,
        'falls': 1.3862943611198906,
        'in': 0.6931471805599453,
        'is': 1.3862943611198906,
        'packs': 1.3862943611198906,
        'snow': 0.28768207245178085,
        'wet': 0.0,
        'winter': 0.6931471805599453})

Write a function that returns a 2D matrix, with a column for each word in the corpus, and a row for each document in the corpus. The values for a row and column will be the tf-idf score for that word in that document.

In [ ]:
def find_tfidf(tf: Dict[str,Dict[str,int]], idf: Dict[str,float], terms: List[str]) -> List[List[float]]:
    N = len(tf)
    tfidf = []
    for c in tf:
        sublist = []
        for i in terms:
            current= tf.get(c)
            if i in current:
                tf_sc = current.get(i)
                idf_sc = idf.get(i)
                score = tf_sc * idf_sc
            else:
                score = 0.0
            sublist.append(score)
        tfidf.append(sublist)
    return tfidf

The function below will print our small Snow matrix in a readable format. 

**Note:** This function will be useless on the larger datasets with many unique words.

In [ ]:
def pretty_matrix(col_head: List[str], row_head: List[str], matrix: List[List[float]]):
    top = "   "
    for t in col_head:
        top += '{:7s}'.format(t)
    print(top)
    for i in range(len(matrix)):
        d = matrix[i]
        s = row_head[i] + ": "
        for f in d:
            s += f'{f:5.3f}' + "  "
        print(s)

In [ ]:
tfidf = find_tfidf(tf, idf, terms)
pretty_matrix(terms, list(filename2document.keys()), tfidf)
tfidf == [[0.0, 0.0, 0.0, 2.772588722239781, 0.6931471805599453, 0.0, 0.0, 0.5753641449035617, 0.0, 0.6931471805599453], 
          [0.6931471805599453, 0.0, 0.28768207245178085, 0.0, 0.6931471805599453, 0.0, 0.0, 0.0, 0.0, 0.6931471805599453], 
          [0.0, 2.772588722239781, 0.28768207245178085, 0.0, 0.0, 0.0, 1.3862943611198906, 0.28768207245178085, 0.0, 0.0], 
          [0.6931471805599453, 0.0, 0.5753641449035617, 0.0, 0.0, 1.3862943611198906, 0.0, 0.28768207245178085, 0.0, 0.0]]

# Part 2 - Distance Matrix and Cosine Similarity

Write a function to calculate the dot product of two vectors $\bf{a}$ and $\bf{b}$.

$$\mathbf{a} \cdot \mathbf{b} = \sum_{i=1}^{n} a_ib_i$$

In [ ]:
def dot_product(a: List[float], b: List[float]) -> float:
    return np.dot(a,b)

In [ ]:
assess(dot_product([1,  2,  3], [4,  8, 12]), 56) and\
assess(dot_product([1,  2,  3], [4,  5,  6]), 32) and\
assess(dot_product([1,  2,  3], [1,  1,  1]),  6) and\
assess(dot_product([1,  2,  3], [3,  2,  1]), 10) and\
assess(dot_product([3,  0,  3], [0,  3,  0]),  0)

Write a function to calculate the cosine similarity of two vectors $\bf{a}$ and $\bf{b}$.

$$cos(\theta) = \frac{\mathbf{a} \cdot \mathbf{b}}{||\mathbf{a}||~||\mathbf{b}||}$$

where 

$$||\mathbf{a}|| = \sqrt{\mathbf{a} \cdot \mathbf{a}}$$

In [ ]:
def cosine_sim(a: List[float], b: List[float]) -> float:
    cosine = np.dot(a,b)
    a = np.dot(a,a)
    b = np.dot(b,b)
    av = math.sqrt(a)
    bv = math.sqrt(b)
    abv = av * bv
    cos = cosine/abv
    return cos
    #for item_a in a:
    #    count = 0
    #    for item_b in b:
    #        over = item_a * item_b
    #        under = (math.sqrt(item_a * item_a) )   
    #for item_b in b:
    #    print('b:')
    #    print(item_b)

In [ ]:
assess(cosine_sim([1,  2,  3], [4,  8, 12]), 1.0) and\
assess(cosine_sim([1,  2,  3], [4,  5,  6]), 0.9746318461970762) and\
assess(cosine_sim([1,  2,  3], [1,  1,  1]), 0.9258200997725515) and\
assess(cosine_sim([1,  2,  3], [3,  2,  1]), 0.7142857142857143) and\
assess(cosine_sim([3,  0,  3], [0,  3,  0]), 0.0)

Write a function to calculate the distance matrix between all row vectors in our tf-idf matrix. The distance should be $1-cosine\_sim$.

In [ ]:
def dist_matrix(vecs: List[List[float]]) -> List[List[float]]:
    dist = []
    c = 0 
    for i in vecs:
        new_list=[]
        d = 0
        for k in vecs:
            total = 1 - cosine_sim(vecs[c], vecs[d])
            new_list.append(total)
            d+=1
        c+=1
        dist.append(new_list)
    return dist

In [ ]:
dmat = dist_matrix(tfidf)
pretty_matrix(list(filename2document), list(filename2document), dmat)
dmat == [[0.0, 0.740251794063685, 0.982331986879515, 0.9670833903997196], 
         [0.740251794063685, 1.1102230246251565e-16, 0.9785579200110728, 0.6881940602419325], 
         [0.982331986879515, 0.9785579200110728, 2.220446049250313e-16, 0.952676589993446], 
         [0.9670833903997196, 0.6881940602419325, 0.952676589993446, -2.220446049250313e-16]]

# Part 3 - Clustering with UPGMA

The UPGMA algorithm enables us to visualize groups of related documents. It is fairly complex to implement, so we will write a series of helper functions for it. 

First, let's write a function to copy a nested list. To copy a regular list in Python, we can slice the whole thing. Run the code block below:

In [ ]:
nums1 = [1, 2, 3, 4]
nums2 = nums1[:]
nums2[0] = 10
print(nums1)
print(nums2)

But if a list is nested, this performs a **shallow copy**:

In [ ]:
m1 = [[1, 2], [3, 4]]
m2 = m1[:]
m2[0][0] = 10
print(m1)
print(m2)

Write a function to create a **deep copy** of a nested list. Assume the list is a matrix - that is, it is only nested at one level of depth.

In [ ]:
def nested_copy(matrix: List) -> List: 
    return [row[:] for row in matrix]

In [ ]:
m1 = [[1, 2], [3, 4]]
m2 = nested_copy(m1)
m2[0][0] = 10
assess(m1, [[1, 2], [3, 4]])
assess(m2, [[10, 2], [3, 4]])

Write a function to calculate the nearest pair of documents when given a matrix of distances between them. The function should return a tuple listing the index of the first document, the index of the second document, and their distance. Ignore all indices in the `prohobited` set.

**Note**: The two documents should be **distinct**.

In [ ]:
def nearest(matrix: List[List[float]], prohibited: Set[int]) -> Tuple[int, int, float]:
    currentlow = 1
    currenti = None
    currentk = None
    c = 0
    for i in matrix:
        d = 0
        for k in matrix[c]:
            if c not in prohibited and d not in prohibited:
                if currenti is None or matrix[c][d] < currentlow:
                    if c != d:
                        currentlow = matrix[c][d]
                        currenti=c
                        currentk=d
            d +=1                
        c+=1
    tuple_data=(currenti, currentk, currentlow)
    return tuple_data

In [ ]:
assess(nearest(dmat, set()), (1, 3, 0.6881940602419325)) and\
assess(nearest(dmat, {1, 3}), (0, 2, 0.982331986879515))

Clusters are defined by a list with 4 values:

* index of first child
* index of second child
* distance between children
* number of documents in the cluster

This is a function to create the list of initial clusters for the leaves in our tree. We have one cluster for each of our original documents.

In [ ]:
def init_clusters(matrix: List[List[float]]) -> List[Tuple[int, int, float, int]]:
    a=0
    cluster_list=[]
    while a < len(matrix): 
        tuple_data=(a, a, 0, 1)
        cluster_list.append(tuple_data)
        a+=1
    return cluster_list

In [ ]:
assess(init_clusters(dmat), [(0, 0, 0, 1), (1, 1, 0, 1), (2, 2, 0, 1), (3, 3, 0, 1)])


Write a function to calculate the unweighted distance of a new cluster formed from $A$ and $B$ to another cluster $X$, when given the size of the clusters $A$ and $B$ and their distances to $X$. This will be used in the UPGMA algorithm below.

$$d_{(A \cup B),X} = \frac{|A|~d_{A,X}~+~|B|~d_{B,X}}{|A|~+~|B|}$$

In [ ]:
def new_dist(a_size: int, b_size: int, d_a_x: float, d_b_x: float) -> float:
    return (abs(a_size)*d_a_x+abs(b_size)*d_b_x)/(abs(a_size+b_size))

In [ ]:
assess(new_dist(1, 1, 0.740, 0.967), 0.8534999999999999) and\
assess(new_dist(2, 1, 0.740, 0.967), 0.8156666666666667)

When `nearest()` finds the two closest clusters and the row indices corresponding to them, we need to create a new row corresponding to the new cluster we get when we combine them.

Given a matrix `m`, row indices `row_a` and `row_b`, and cluster sizes `a_size` and `b_size`, return a list representing a new row of `m`. Do **not** modify `m` in this function; just generate the new row list.

To generate the new row list:
* For every cluster *X* in the matrix:
  * Call `new_dist()` to find $$d_{(A \cup B),X}$$
  * Add this distance to the new row.
* Add an entry of *0.0* to represent the distance from the new cluster to itself.

In [ ]:
def create_new_row(m: List[List[float]], row_a: int, row_b: int, a_size: int, b_size: int):
    new_list=[]
    for i in range(0, len(m)):
        d_a_x = m[row_a][i]
        d_b_x = m[row_b][i]
        n_dist = new_dist(a_size, b_size, d_a_x, d_b_x)
        new_list.append(n_dist)
    new_list.append(0.0)
    return new_list

In [ ]:
new_row = create_new_row(nested_copy(dmat), 1, 3, 1, 1)
assess(new_row, 
       [0.8536675922317023, 0.3440970301209663, 0.9656172550022594, 0.34409703012096615, 0.0])

Given a row for a new cluster `X` created by `create_new_row()`, we now want to add it back to the original matrix. This requires not only appending the row to the matrix, but also adding a column to each existing row to keep the matrix symmetric.

In [ ]:
def add_row_to(m: List[List[float]], new_row: List[float]):
    m.append(new_row)
    l=len(m)-1
    for i in range(0, len(m)-1):
        m[i].append(0.0)
        m[i][l]=m[l][i]

In [ ]:
m_copy = nested_copy(dmat)
add_row_to(m_copy, new_row)
labels = list(filename2document) + ['X']
pretty_matrix(labels, labels, m_copy)
m_copy == [[0.0, 0.740251794063685, 0.982331986879515, 0.9670833903997196, 0.8536675922317023],
           [0.740251794063685, 1.1102230246251565e-16, 0.9785579200110728, 0.6881940602419325, 0.3440970301209663],
           [0.982331986879515, 0.9785579200110728, 2.220446049250313e-16, 0.952676589993446, 0.9656172550022594],
           [0.9670833903997196, 0.6881940602419325, 0.952676589993446, -2.220446049250313e-16, 0.34409703012096615],
           [0.8536675922317023, 0.3440970301209663, 0.9656172550022594, 0.34409703012096615, 0.0]]

Finally, write a function that brings in a distance matrix and creates clusters using the UPGMA algorithm. This algorithm  iteratively combines the two closest clusters until there is one single tree. Here is the pseudocode for this algorithm:

* Create the initial list of leaf clusters
* While we still have multiple independent clusters
  * Find the nearest two clusters according to our distance matrix
  * Join these two clusters to make a new cluster
  * Add new distances to our matrix for every row and column between it and the new cluster
* Return the list of created clusters, minus the initial leaves

In [ ]:
def upgma(m: List[List[float]]) -> List[Tuple[int, int, float, int]]:
    m_copy = nested_copy(m)
    list_of_tuples = init_clusters(m_copy)
    current_clusters = []
    exclude=set()
    i=0
    while i < len(m)-1:
        closest=nearest(m_copy, exclude) #c[0] = vec1, c[1] = vec2, c[2] = distance
        exclude.add(closest[0])
        exclude.add(closest[1])
        new_row =create_new_row(m_copy, closest[0], closest[1], list_of_tuples[closest[0]][3], list_of_tuples[closest[1]][3])
        add_row_to(m_copy, new_row)
        sum1=list_of_tuples[closest[1]][3]+list_of_tuples[closest[0]][3]
        current_clusters.append((closest[0], closest[1], closest[2], sum1))
        list_of_tuples.append((closest[0], closest[1], closest[2], sum1))
        i+=1
    return current_clusters

In [ ]:
assess(upgma(dmat), 
       [(1, 3, 0.6881940602419325, 2),
        (0, 4, 0.8536675922317023, 3),
        (2, 5, 0.9711888322946779, 4)])

# Part 4 - Dendrogram

Finally, we can use the $dendrogram$ function in the $scipy.hierarchy$ module to draw the resulting tree from our clustered data.

In [ ]:
def tf_only_matrix(tf: Dict[str,Dict[str,int]]) -> List[List[float]]:
    return [[counts.get(term, 0) for term in terms] 
            for filename, counts in tf.items()]

def plot_dendrogram(filename2document: Dict[str,str], title: str, ignore_idf=False):
    tf = term_freqs(filename2document)
    terms = all_terms(filename2document)
    idf = inv_doc_freqs(filename2document, terms)
    if ignore_idf:
        c = upgma(dist_matrix(tf_only_matrix(tf)))
    else:
        c = upgma(dist_matrix(find_tfidf(tf, idf, terms)))
    plt.figure()
    plt.figure(figsize=(10, 10))
    dn = hierarchy.dendrogram(c, labels = list(filename2document), leaf_rotation=90)
    plt.title(f"UPGMA Clustering: {title}")
    plt.ylabel("total distance")
    plt.show()

In [ ]:
plot_dendrogram(filename2document, "Winter Sentences using tf-idf")

# Part 5 - Experiments and Discussion

Draw a dendrogram and discuss the results of your clustering for each of the following four experiments
* Poems using tf only
* Poems using tf-idf
* Books using tf only
* Books using tf-idf

In [ ]:
def tf_only_matrix(tf: Dict[str,Dict[str,int]]) -> List[List[float]]:
    return [[counts.get(term, 0) for term in terms] 
            for filename, counts in tf.items()]

In [ ]:
poems = file_dictionary('/kaggle/input/csci-270-poems-2022')

plot_dendrogram(poems, "Poems, TF only", True)
plot_dendrogram(poems, "Poems, TF-IDF")



## Poem Discussion

1. Compare and contrast the TF-only clustering with the TF-IDF clustering. Which corresponds better to your intuition about what makes the poems similar? Why?
    
    I expected the graphs to cluster the books in a similiar order in both graphs. Instead they are compared in various different orders. I expected them to be less similiar in the IDF graph because of the lack of meaningful words. 



2. Carefully read two poems that are relatively close to your poem in the TF-IDF clustering, and two poems that are distant from your poem. What specific aspects of these poems do you think are represented well by the TF-IDF concept? What are some aspects that TF-IDF overlooks?

    (My poem is The road not taken, I compared it to (close) Things/Viva La Vida, and (distant)ThunderRoad/Misery Business)
    I think that the tf-idf clustering is able to capture the overall theme of a poem well. It seems like the poems more similiar to mine were talking about similiar subjects. Specifically personifying things or events, and longing for the past, or to change it. The poems that were more closely related to my poem were also just more alike in overall structure where the more distant poems had a quite different structure. 
    


In [ ]:
books = file_dictionary('/kaggle/input/csci-270-books-2022')

plot_dendrogram(books, "Books, TF only", True)
plot_dendrogram(books, "Books, TF-IDF")

## Book Discussion

1. Compare and contrast the TF-only clustering with the TF-IDF clustering. Which corresponds better to your intuition about what makes the books similar? Why?

    The Tf-clustering seems to be more inline with what my assumptions were. It seems books dealing heavily with adventure were clustered much more appropriately. At least it makes sense to me that 'King arthur', 'The Lion The Witch and The Wardrobe', & 'Robin Hood' are more similiar to huck-finn than the great gatsby. 

2. Skim a few pages of two books that are relatively close to your book in the TF-IDF clustering, and two books that are distant from your book.  What specific aspects of these books do you think are represented well by the TF-IDF concept? What are some aspects that TF-IDF overlooks?

    It makes sense that treasure island would be closely paired with huck-finn. Not only because they both have characters named jim, but because their plots both rely heavily the adventuring the characters do. The next cloely related book is the great gatsby. I cannot think of any particular reason why this would be so closely related to huck finn. I feel like they deal with very different topics. It does makes sense that books like darth plagueis and do androids dream of sheep, are clustered away from huck-finn because their science fiction nature.

## Chapter Analysis

Break up your book into a dictionary where the keys are chapter names or numbers and the values are the contents of each chapter (or other pertinent subdivision).  Then apply `plot_dendrogram()`. Examine the dendrogram, and then answer the questions below.

In [ ]:
## Write code here to create the Chapter Analysis dendrogram.
chapters = books['huck-finn_fixed.txt'].split('CHAPTER')

dis = {i: chapters[i] for i in range(1, len(chapters))}
plot_dendrogram(dis, "Huckleberry Finn")

## Chapter Analysis Questions

1. Examine the various clusters of chapters. To what extent does the similarity among chapters correspond to different stages of the story and plot?

      The two most similiar chapters are 8, and 14. chapter 8 is when huck and jim meet up to begin their adventure. There is tension between the two of them and huck has a moral delema as to whether or not to turn jim in. Chapter 14 is huck retelling a bible story about king solomon to jim. The next closest related is chapter 18, which deals with the two feuding families huck and jim get caught between. They have to escape the battle that happens between the groups. The next closest chapter is 15, where Huck and jim lose each other on the river. Chapter 23 is next & deals with the two hobos who pretend to be royalty on the raft. Chapter 4 is the only particular outlier in this group. It takes place before the river adventure starts, which seems to be the only commonality between these other chapters. It deals with Huckleberry giving away his fortune because he fears his father will come back and take it from him. The final closely related chapter is 16. In this chapter Huck has to sneak jim past patrols/cities of slave search parties. I cannot discern a single plot similiarity between all of these chapters. It just seems that most of them are indivudual adventures that take place on the river.  

2. Does this clustering of chapters suggest to you any new insights about the book? Explain.

    Nothing particular stands out from these chapters. They all seem to deal with heavy/intense subjects on their own, but none of their dramas are related. The only thing I can think of is that they are all stressful adventures by themselves.